In [58]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt

#Hypothesis: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)

#The following data files are available for this assignment:
#From the Zillow research data site there is housing data for the United States. In particular the datafile for all homes at a city level, City_Zhvi_AllHomes.csv, has median home sale prices at a fine grained level.
#From the Wikipedia page on college towns is a list of university towns in the United States which has been copy and pasted into the file university_towns.txt.
#From Bureau of Economic Analysis, US Department of Commerce, the GDP over time of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file gdplev.xls. For this assignment, only look at GDP data from the first quarter of 2000 onward.

# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

df = pd.read_excel('gdplev.xls')
to_drop = [df.columns[3], df.columns[7]]
df.drop(to_drop, inplace=True, axis='columns')
df.columns = ['Years', 
              'GDP in billions of current dollars(annual)', 
              'GDP in billions of chained 2009 dollars(annual)',
              'Quarters',
              'GDP in billions of current dollars(quarter)',
              'GDP in billions of chained 2009 dollars(quarter)']

# plt.plot(df.iloc[219:,3],df.iloc[219:,5])
# plt.show()

req_df = df.iloc[219:, [3,5]].reset_index(drop=True)

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns = []

    with open('university_towns.txt') as file:
        for line in file:
            if '[edit]' in line:
                state = line[:line.find('[')]
            else:
                regionName = line[:line.find(' (')]
                university_towns.append([state, regionName])

    dataframe = pd.DataFrame(university_towns, columns=['State', 'RegionName'])
    
    return dataframe

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    for index, row in req_df.iterrows():
        if index >= req_df.shape[0]-2:
            break
        if row[1]>req_df.iloc[index+1, 1] and req_df.iloc[index+1, 1]>req_df.iloc[index+2, 1]:
            return row[0]
    
    return 

def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    for index, row in req_df.iterrows():
        if index >= req_df.shape[0]-2:
            break
        if row[1]<req_df.iloc[index+1, 1] and req_df.iloc[index+1, 1]<req_df.iloc[index+2, 1] and req_df.iloc[index+2, 0] > get_recession_start():
            return req_df.iloc[index+2, 0]
    
    return

def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    recession_index = req_df.index[req_df['Quarters']==get_recession_end()][0]
    recession_bottom = req_df.iloc[recession_index-2, 0]

    return recession_bottom

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.'''
#     Reading csv file
    df = pd.read_csv('City_Zhvi_AllHomes.csv', encoding="latin")
#     Removing unnecessary columns
    df.drop(df.columns[0] , axis=1, inplace=True)
    df.drop(df.columns.to_series()["Metro":"1999-12"] , axis=1, inplace=True)
    df.drop(df.columns.to_series()["2016-10":"2019-04"] , axis=1, inplace=True)
#     Multi-Indexing
#     df = df.groupby(['State', 'RegionName']).mean()
    df = df.set_index(['State', 'RegionName'])
    df = df.sort_values(['State', 'RegionName'])
#     Converting housing data in quarters with mean values
    new_columns = [str(x)+y for x in range(2000, 2017) for y in ['q1','q2','q3','q4']]
    
#     NOTE: This function isn't complete yet. Little more code has to be added.
    
    return df

def run_ttest():
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start = get_recession_start()
    bottom = get_recession_bottom()
    housing_data = convert_housing_data_to_quarters()
    housing_data = housing_data.loc[:, start: bottom]
    housing_data.reset_index(inplace=True)
    
    
    housing_data['price_ratio'] = (housing_data[start] - housing_data[bottom]) / housing_data[start]
    
    uni_towns = get_list_of_university_towns()  
    uni_town_list = uni_towns['RegionName'].tolist()
    
    housing_data['isUniTown'] = housing_data.RegionName.apply(lambda x: x in uni_town_list)
    
    uni_data = housing_data[housing_data.isUniTown].copy().dropna()
    not_uni_data = housing_data[~housing_data.isUniTown].copy().dropna()

    p = ttest_ind(uni_data['price_ratio'], not_uni_data['price_ratio'])[1]
    different = p < 0.01   
    
    lower = 'university town' if uni_data['price_ratio'].mean() < not_uni_data['price_ratio'].mean() else 'non-university town'

    return (different, p, lower)

run_ttest()
    
    return

convert_housing_data_to_quarters()

['2000q1', '2000q2', '2000q3', '2000q4', '2001q1', '2001q2', '2001q3', '2001q4', '2002q1', '2002q2', '2002q3', '2002q4', '2003q1', '2003q2', '2003q3', '2003q4', '2004q1', '2004q2', '2004q3', '2004q4', '2005q1', '2005q2', '2005q3', '2005q4', '2006q1', '2006q2', '2006q3', '2006q4', '2007q1', '2007q2', '2007q3', '2007q4', '2008q1', '2008q2', '2008q3', '2008q4', '2009q1', '2009q2', '2009q3', '2009q4', '2010q1', '2010q2', '2010q3', '2010q4', '2011q1', '2011q2', '2011q3', '2011q4', '2012q1', '2012q2', '2012q3', '2012q4', '2013q1', '2013q2', '2013q3', '2013q4', '2014q1', '2014q2', '2014q3', '2014q4', '2015q1', '2015q2', '2015q3', '2015q4', '2016q1', '2016q2', '2016q3', '2016q4']
